numpy and scipy and how to get the saturation point from a cubic EoS
==
This notebook intends to present van der Waals's Equation of State([wikipedia](https://en.wikipedia.org/wiki/Van_der_Waals_equation),[nobel prize lecture](http://www.nobelprize.org/nobel_prizes/physics/laureates/1910/waals-lecture.pdf)), and to derive get density for given values of Pressure and Temperature.

> **Here you will meet the following topics:**
>- Thermodynamics: `equation of state`, `density`,  `saturation point`
>- Python: `packages`, `numpy`, `scipy`

Here we will use some resources of the packages numpy and scipy

To import a module into our notebook we use the following syntax:

In [1]:
import numpy as numpy #this line imports the module numpy
#keyword import
       #name of the module
             #keyword "as"
                #alias name

the following slightly different syntax works similarly:

In [2]:
from scipy import optimize as optimize
#keyword "from"
     #name of the parent module
           #keyword "import"
                  #name of the desired subpackage
                           #keyword "as"
                              #alias name

to use some function of this module we have to write 'alias name'.'function name'
as an example we will see below:
numpy.roots, numpy.log, numpy.where, optimize.bissect

Supose you have to solve the following problem in a thermodynamics course:
>"Calculate saturation pressure for pure hexane at temperature of 220 K according to Peng-Robinson equation of state"

We will show how here to implement a function to calculate density of a pure fluid at given T and P; how calculate its fugacity coefficnient at that condition, and how to find saturaton pressure at given temperature, to according to Peng-Robinson equation of state.

Here we start by assigning value to some thermodynamic variables

In [3]:
#universal gas constant
R = 0.08314 #CHECK UNITS

#pure component critical parameters
## Tc (K)
Tc = 507.5

## Pc (bar)
Pc = 30.1

##accentric factor
ac=0.299

In [4]:
#calculate saturation pressure at what temperature?
T = 220.

[ ] Write references

In [7]:
### Calculate Psat using the algorith from

#[ ] Write relevant equation in mathjax format

#calculate the coeficients of the EoS from the critical properties
kappa = 0.37464+1.54226*ac-0.26992*ac*ac   
a = 0.45724*R*R*Tc*Tc/Pc
teta = a*(1.+kappa-kappa*(T/Tc)**0.5)**2 
b = 0.0778*R*Tc/Pc
sig = 1. + 2**0.5
eps = 1. - 2**0.5
dteta = kappa*(-kappa+(1+kappa)*(T/Tc)**0.5)*-a/Tc

#[ ] write dP/dV=0 in mathjax
#[ ] write polynomial-in-V representation of dP/dV=0 in mathjax

#calculate each coeffcient of the polynomial-in-V representation of dP/dV=0
rho = R*T
mu = 2*(b*R*T*(eps+sig)-teta)
nu = R*T*(b**2)*(4*sig*eps+eps*eps+sig*sig) - b*(teta*(eps+sig-4))
omega=2*(b**3)*R*T*(sig*eps*eps+eps*sig*sig)- 2*(b**2)*(teta*(1-sig-eps))
zeta = R*T*(b**4)*sig*sig*eps*eps-teta*(b**3)*(eps+sig)

#use the method "roots" from package "numpy" to calculate the roots of tyhe polynomial and assing the solutions to the variable plateaus as an array
plateaus = numpy.roots([rho,mu,nu,omega,zeta])

print("plateaus:", plateaus)


#filter out values of plateu correspondign to non-physical volume (less than b)

plateaus = plateaus[ numpy.where( plateaus > b ) ]

print("plateaus:", plateaus)

Vli = min(plateaus)
Vvi = max(plateaus)

#find the corrsponding values of pressure for each filtered volume

Psupi = R*T/(Vvi-b)-teta/((Vvi+sig*b)*(Vvi+eps*b))-0.000000001
Pinf = R*T/(Vli-b)-teta/((Vli+sig*b)*(Vli+eps*b))

#note that the algorithm cannot work with pressure less than or equal to zero, therefore fix minimium pressure to slightly above zero if is resulted negative

if Pinf > 0:
    Pinfi = Pinf+0.000000001
else:
    Pinfi = 0   +0.000000001

print("Vli:",Vli)
print("Vvi:",Vvi)
print("Psupi:",Psupi)
print("Pinfi:",Pinfi)     

#[ ] write reference to this equaiton in mathjax

#define function for usage internal to the algorith
def psia(V):
    return (1/(b*(eps-sig)))*numpy.log((V+eps*b)/(V+sig*b))

#[ ] write reference to this equaiton in mathjax

#define equilibrium criteria
def fps(P):
    alfa = (sig+eps-1)*b-R*T/P
    beta = sig*eps*b**2-(b+R*T/P)*(sig+eps)*b + teta/P
    gamma = -(b+R*T/P)*(sig*eps*b**2)-b*teta/P
    V = numpy.roots([1,alfa,beta,gamma])
    Vl = min(V)
    Vv = max(V)
    return (P*(Vl-Vv) + (T*dteta-teta)*(psia(Vl)-psia(Vv))+T*(R*numpy.log((Vv-b)/(Vl-b))+dteta*(psia(Vv)-psia(Vl))))/(R*T)

#ask package optmize to use method bissect to solve the equilibrium criteria just defined

psat = optimize.bisect(fps, Pinfi, Psupi, xtol=1e-12, rtol=4.4408920985006262e-16, maxiter=100, full_output=True, disp=True)

print("Saturation pressure = [",str(psat[0]),"] bar")



plateaus: [ 4.25591143 -0.10653629  0.14642616  0.09212116]
plateaus: [ 4.25591143  0.14642616]
Vli: 0.14642616259
Vvi: 4.25591142877
Psupi: 2.09228234169
Pinfi: 1e-09
Saturation pressure = [ 0.001735215064619947 ] bar


# External references
scipy, optimize [documentation](http://docs.scipy.org/doc/scipy/reference/optimize.html)

# Credits
* Initially developed in python 2.7 by Guilherme Carneiro Queiroz da Silva
* Adapted to ipynb with python 3.5 by Iuri Soter Viana Segtovich